In [22]:
from __future__ import print_function

import numpy as np
import pickle
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder

from keras.preprocessing import sequence
from keras.layers.noise import GaussianNoise
from keras.models import Sequential
from keras.layers.core import Dense,Dropout,Activation,Flatten,Reshape
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Convolution2D,MaxPooling2D
from keras.utils import np_utils

In [13]:
#set parameters

maxlen = 100 #?
batch_size = 50
nb_filter = 64
filter_length_1 = 10
filter_length_2 = 7
hidden_dims = 10
nb_epoch = 30
nb_classes = 16

In [6]:
#load the MFCC data
mfcc_filename = "mfccs_dev_22050.csv"
dataset = pd.read_csv(mfcc_filename)
df = pd.DataFrame(dataset)
mfcc_data = df[["mfcc_mean1","mfcc_mean2","mfcc_mean3","mfcc_mean4","mfcc_mean5","mfcc_mean6","mfcc_mean7","mfcc_mean8","mfcc_mean9","mfcc_mean10","mfcc_mean11","mfcc_mean12","mfcc_mean13"]]
test_dim = mfcc_data.shape[0]

#set up targets (labels)
accent_data = pd.read_csv("cv-valid-dev-acc-mp3.csv")
targets_raw = np.array(accent_data['accent'])

label_encoder = LabelEncoder()
mfcc_targets = label_encoder.fit_transform(targets_raw)

X_train,X_test,y_train,y_test = train_test_split(
    mfcc_data,mfcc_targets,test_size=0.20,random_state=42
)

Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)

X_train = X_train.values.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.values.reshape(X_test.shape[0], X_test.shape[1],1)

print(len(X_train), 'train sequences')
print(len(X_test), 'test sequences')
print('X_train shape:', X_train.shape)
print('X_test shape:', X_test.shape)
print('y_train shape:', y_train.shape)
print('y_test shape:', y_test.shape)

1080 train sequences
270 test sequences
X_train shape: (1080, 13, 1)
X_test shape: (270, 13, 1)
y_train shape: (1080,)
y_test shape: (270,)


In [30]:
#build model
model = Sequential()

# model.add(Reshape((X_train.shape[1], X_train.shape[1], 1)))

#layer 1
#add a Convolution2D which will learn nb_filter mfcc groups:
model.add(Convolution2D(nb_filter=nb_filter,
                        kernel_size=filter_length_1,
                        input_shape=(X_train.shape[1], X_train.shape[1],1),
                        padding='valid', activation='relu'))

#batch normalization to keep weights in [0,1] range:
model.add(BatchNormalization())

#layer2
model.add(Convolution2D(nb_filter=nb_filter,
                        kernel_size=filter_length_2,
                        padding='same', activation='relu'))
model.add(BatchNormalization())

#standard maxpooling to halve the output of previous layer
model.add(MaxPooling2D(pool_size=(2,2)))

#layer3
model.add(Convolution2D(nb_filter=nb_filter,
                        kernel_size=filter_length_2,
                        padding='same', activation='relu'
                        ))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))

# Dropout reduces overfitting
model.add(Dropout(.25))

#flatten the output of the convolutional layers
#so that we can add a vanilla dense layer
model.add(Flatten())

#project onto a single unit output layer
#and squash with a softmax into 0-1 probability space
model.add(Dense(nb_classes, activation='softmax'))

#fit model
model.compile(loss='categorical_crossentropy',
              optimizer='adam',metrics=["accuracy"])
model.fit(X_train, Y_train, batch_size=batch_size, nb_epoch=nb_epoch, verbose=1)
model.summary()

#print report of recall, precision, f1 score
y_pred = model.predict_classes(X_test)
print(classification_report(y_test,y_pred))

/Users/yeojin_jung/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(kernel_size=10, input_shape=(13, 13, 1..., padding="valid", activation="relu", filters=64)`
  # This is added back by InteractiveShellApp.init_path()
/Users/yeojin_jung/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:19: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(kernel_size=7, padding="same", activation="relu", filters=64)`
/Users/yeojin_jung/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:28: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(kernel_size=7, padding="same", activation="relu", filters=64)`
/Users/yeojin_jung/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:47: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


ValueError: Error when checking input: expected conv2d_34_input to have 4 dimensions, but got array with shape (1080, 13, 1)